In [1]:
import snowflake.connector
import json
from datetime import datetime

In [2]:
def json_serializer(obj):
    """Custom serializer for non-JSON-native objects"""
    if isinstance(obj, datetime):
        return obj.isoformat()
    raise TypeError(f"Type {type(obj)} not serializable")

In [ ]:
# Snowflake connection details
import configparser
 
config = configparser.ConfigParser()
config.read('config.ini')
 
SNOWFLAKE_USER = config['snowflake']['user']
SNOWFLAKE_PASSWORD = config['snowflake']['password']
SNOWFLAKE_ACCOUNT = config['snowflake']['account']
SNOWFLAKE_WAREHOUSE = config['snowflake']['warehouse']
SNOWFLAKE_DATABASE = config['snowflake']['database']
SNOWFLAKE_SCHEMA = config['snowflake']['schema']
 
conn = snowflake.connector.connect(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account=SNOWFLAKE_ACCOUNT,
    warehouse=SNOWFLAKE_WAREHOUSE,
    database=SNOWFLAKE_DATABASE,
    schema=SNOWFLAKE_SCHEMA
)
 

In [4]:
# Fetch cleaned incident data
cursor = conn.cursor()
cursor.execute("SELECT * FROM cleaned_alerts")  # Your cleaned table
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]

In [5]:
# Convert to JSON with datetime handling
incident_data = [dict(zip(columns, row)) for row in rows]
incident_json = json.dumps(incident_data, 
                          default=json_serializer,  # Add custom serializer
                          indent=4)

In [6]:
# Save JSON to file
with open("incident_data.json", "w") as f:
    f.write(incident_json)

In [7]:
# Close connections
cursor.close()
conn.close()